In [1]:
import pandas as pd
import numpy as np

import os

from load_wastewater_data import *
from plotting_tools import *
import seaborn as sns

# Update rcParams to set the default font to Times New Roman
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'DejaVu Serif'

In [2]:
# define experiment to be preprocessed
experiment_series = "2024_09_17"

resolution = "3_min_resolution" 
degradation_setting = "no_decay" # one of "no_decay", "linear_decay_0_0001", "constant_decay_dynamics"

In [3]:
plot_path = f"../plots/wastewater_model/rain_scenarios"
os.makedirs(os.path.join(plot_path), exist_ok=True)
file_name = f"concentrations_{resolution}_{degradation_setting}"

Load data

In [4]:
rain_scenarios = ["KeinRegen", "Nieselregen", "MittelstarkerRegen"]

# Load data
df_1 = load_systems_data(experiment_series, resolution, rain_scenarios[0], degradation_setting, file_type="concentrations")
df_2 = load_systems_data(experiment_series, resolution, rain_scenarios[1], degradation_setting, file_type="concentrations")
df_3 = load_systems_data(experiment_series, resolution, rain_scenarios[2], degradation_setting, file_type="concentrations")

df_1["rain_scenario"] = rain_scenarios[0]
df_2["rain_scenario"] = rain_scenarios[1]
df_3["rain_scenario"] = rain_scenarios[2]
df_rain = pd.concat([df_1, df_2, df_3])

## Visualizations

#### plot rain scenarios next to each other

In [5]:
def rain_scenarios_next_to_each_other(station):
    df_station = df_rain.loc[df_rain.sampling_point==station,:].sort_values("time_in_days")

    fig, axs = plt.subplots(ncols=3, sharey=True, sharex=True, figsize=(15.5, 3), dpi=300)
    for i, (variate, variate_title) in enumerate(zip(["KeinRegen", "Nieselregen", "MittelstarkerRegen"], ["no precipitation", "moderate gentle rain showers", "moderate rain showers"])):
        sns.lineplot(data=df_station.loc[df_station.rain_scenario==variate,:], x="time_in_days", y="COVID_copies/l", ax=axs[i], color=["slategray", "cornflowerblue", "navy"][i])
        axs[i].set_title(f"{variate_title}")    
        axs[i].set_xlabel(f"simulation time [days]")
        if i==0:
            axs[i].set_ylabel(f"virus levels\n[copies/l]")
        else:
            axs[i].set_ylabel("")
        axs[i].tick_params(axis='x')
        axs[i].tick_params(axis='y')
        axs[i].set_ylim(0,430)

    axs[0].set_xticks(np.arange(0, 15, 2))
    
    plt.tight_layout()
    fig.savefig(os.path.join(plot_path, file_name + f"_station_{station}.png"))
    plt.close(fig)



for station in ["16"]:
    rain_scenarios_next_to_each_other(station)

#### Plot rain scenarios on top of each other

In [ ]:
def plot_rain_scenarios(df_rain, station, save_fig=False):
    fig, ax = plt.subplots(figsize = (12,6)) 
    df_sub = df_rain.loc[df_rain.sampling_point==station, :].sort_values("time_in_days")
    n_nans = df_sub.isna().sum()["COVID_copies/l"]
    print(f"{station}: {n_nans} NaNs detected, {df_sub.shape[0]} samples in total")
    if n_nans>0:
        print("Dropping NaNs")
        df_sub = df_sub.dropna()
    sns.lineplot(df_sub, x="time_in_days", y="COVID_copies/l", hue="rain_scenario")
    plt.title(f"{station}")
    plt.xlabel('simulation time [days]')
    plt.ylabel('virus levels [copies/l]')
    plt.close(fig)
    return fig

In [ ]:
plot_rain_scenarios(df_rain, "16")